## Loading the library

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Sequential

In [13]:
# !pip install opencv-python

## Initializing paramaters

In [10]:
IMG_SHAPE = (64, 64, 3)
CODE_SIZE = 100  

In [13]:
class GAN:

    def __init__(self, img_shape, code_size):
        self.global_epoch = 0
        self.IMG_SHAPE = img_shape
        self.CODE_SIZE = code_size
        self.data = None
        self.generator = None
        self.discriminator = None

    def get_data(self, video_file='data/video.avi', frame_skip=5):
        print('Generating data from video.')
        vidcap = cv2.VideoCapture(video_file)
        success, image = vidcap.read()
        count = 0
        data = []

        while success:
            y_dim, x_dim = image.shape[:2]
            dim_diff = x_dim - y_dim
            x_start = np.random.randint(dim_diff)
            image = image[:, x_start:x_start + y_dim, :]
            image = cv2.resize(image, self.IMG_SHAPE[:2])
            image = image[:, :, ::-1] 

            if count % frame_skip == 0:
                data.append(image)
            count += 1
            success, image = vidcap.read()

        self.data = np.asarray(data) / 255.0
        print(f'Data prepared. {len(data)} frames generated.')

  ## Creating my DCGANs conv layer
    def create_generator(self):
        self.generator = Sequential([
            L.Input(shape=(self.CODE_SIZE,)),
            L.Dense(16 * 16 * 128, activation='elu'),  
            L.Reshape((16, 16, 128)),
            L.Conv2DTranspose(128, kernel_size=(5, 5), strides=2, padding='same', activation='elu'),  ## Maine yaha elu use kiya because of its higher cal variance
            L.Conv2DTranspose(64, kernel_size=(5, 5), strides=2, padding='same', activation='elu'),
            L.Conv2D(3, kernel_size=(7, 7), padding='same', activation='sigmoid')  # Final output
        ])
        print('Generator created successfully.')

    def create_discriminator(self):
        self.discriminator = Sequential([
            L.Input(shape=self.IMG_SHAPE),
            L.Conv2D(64, kernel_size=(5, 5), strides=2, padding='same', activation='elu'),
            L.Conv2D(128, kernel_size=(5, 5), strides=2, padding='same', activation='elu'),
            L.Flatten(),
            L.Dense(1, activation='sigmoid')  
        ])
        print('Discriminator created successfully.')

    def compile_models(self):
        self.discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        print('Models compiled successfully.')

    def sample_noise_batch(self, bsize):
        return np.random.normal(size=(bsize, self.CODE_SIZE)).astype('float32')

    def sample_data_batch(self, bsize):
        idxs = np.random.choice(np.arange(self.data.shape[0]), size=bsize)
        return self.data[idxs]

    def sample_images(self, nrow, ncol, epoch, sharp=False):
        images = self.generator.predict(self.sample_noise_batch(bsize=nrow * ncol))
        images = np.clip(images, 0, 1)

        for i in range(nrow * ncol):
            plt.subplot(nrow, ncol, i + 1)
            plt.imshow(images[i].reshape(self.IMG_SHAPE), cmap="gray", interpolation="kaiser")
            plt.xticks([])
            plt.yticks([])
        plt.suptitle('Epochs: ' + str(epoch))

        sample_dir = 'output'
        os.makedirs(sample_dir, exist_ok=True)
        plt.savefig(os.path.join(sample_dir, f'render_epochs_{epoch}.png'), dpi=200)
        plt.close()

    def train(self, epochs, batch_size, discriminator_steps=5, generator_steps=1):
        for epoch in range(epochs):
            real_batch = self.sample_data_batch(batch_size)
            noise_batch = self.sample_noise_batch(batch_size)

            # Train discriminator
            for _ in range(discriminator_steps):
                d_loss_real = self.discriminator.train_on_batch(real_batch, np.ones((batch_size, 1)))
                generated_images = self.generator.predict(noise_batch)
                d_loss_fake = self.discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))

            # Train generator
            g_loss = self.discriminator.train_on_batch(self.generator.predict(noise_batch), np.ones((batch_size, 1)))

            if epoch % 100 == 0:
                self.sample_images(4, 5, self.global_epoch, sharp=True)
                print('Images successfully generated.')

            if epoch % 1 == 0:
                print(f'Epoch: {self.global_epoch}, Discriminator loss: {d_loss_real[0]:.4f}, Generator loss: {g_loss[0]:.4f}')

            self.global_epoch += 1

In [ ]:
gan = GAN(IMG_SHAPE, CODE_SIZE)
gan.get_data(video_file='data/video.avi', frame_skip=5)
gan.create_generator()
gan.create_discriminator()
gan.compile_models() 
gan.train(epochs=10000, batch_size=100, discriminator_steps=5, generator_steps=1)


Generating data from video.
Data prepared. 6630 frames generated.
Generator created successfully.
Discriminator created successfully.
Models compiled successfully.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Images successfully generated.
Epoch: 0, Discriminator loss: 1.2811, Generator loss: 1.2280
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Epoch: 1, Discriminator loss: 0.9880, Generator loss: 0.9830
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

In [54]:
# # start the process
# gan = GAN(IMG_SHAPE,CODE_SIZE)
# gan.get_data(video_file='data/video.avi', frame_skip=5)
# gan.create_generator()
# gan.create_discriminator()
# gan.create_tf_objects()
# gan.train(epochs=100000, batch_size=100, discriminator_steps=5, generator_steps=1)